# Assignment 2 - Task 1: Model Building

This notebook builds and evaluates two deep learning models for monthly retail turnover forecasting:
- Fully-connected deep neural network (tabular baseline)
- GRU-based sequence-to-sequence recurrent network

## Setup

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    r2_score,
)

import matplotlib.pyplot as plt
import seaborn as sns

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

AUTOTUNE = tf.data.AUTOTUNE
from google.colab import drive
drive.mount('/drive')
DATA_DIR = Path('/drive/My Drive/Colab Notebooks/notebooks/data/processed')

## Load processed datasets

In [ ]:
train_df = pd.read_csv(DATA_DIR / 'train.csv', parse_dates=['Month'])
val_df = pd.read_csv(DATA_DIR / 'val.csv', parse_dates=['Month'])
test_df = pd.read_csv(DATA_DIR / 'test.csv', parse_dates=['Month'])

print('Shapes -> train:', train_df.shape, 'val:', val_df.shape, 'test:', test_df.shape)
train_df.head()

## Feature engineering and shared preprocessing

In [ ]:
def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    df['MonthSin'] = np.sin(2 * np.pi * df['MonthNum'] / 12.0)
    df['MonthCos'] = np.cos(2 * np.pi * df['MonthNum'] / 12.0)
    return df

for frame in (train_df, val_df, test_df):
    add_time_features(frame)

all_series_ids = pd.concat([
    train_df['Series ID'],
    val_df['Series ID'],
    test_df['Series ID'],
], axis=0).unique()
series_ids = sorted(all_series_ids)
series_index = {sid: idx for idx, sid in enumerate(series_ids)}
denominator = max(len(series_ids) - 1, 1)

for frame in (train_df, val_df, test_df):
    frame['SeriesIndex'] = frame['Series ID'].map(series_index).astype('float32')
    frame['SeriesIndexNorm'] = frame['SeriesIndex'] / denominator

turnover_scaler = StandardScaler()
year_scaler = StandardScaler()

train_df['TurnoverScaled'] = turnover_scaler.fit_transform(train_df[['Turnover']])
val_df['TurnoverScaled'] = turnover_scaler.transform(val_df[['Turnover']])
test_df['TurnoverScaled'] = turnover_scaler.transform(test_df[['Turnover']])

train_df['YearScaled'] = year_scaler.fit_transform(train_df[['Year']])
val_df['YearScaled'] = year_scaler.transform(val_df[['Year']])
test_df['YearScaled'] = year_scaler.transform(test_df[['Year']])

train_df[['State', 'Industry', 'Series ID', 'Turnover', 'Month', 'MonthSin', 'MonthCos']].head()

## Model 1: Deep Neural Network (tabular)

In [ ]:
target_col = 'Turnover'
dnn_categorical_cols = ['State', 'Industry', 'Series ID']
dnn_numeric_cols = ['Year', 'MonthNum', 'Quarter', 'MonthSin', 'MonthCos']
dnn_feature_cols = dnn_categorical_cols + dnn_numeric_cols

dnn_preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore', sparse_output=False), dnn_categorical_cols),
        ('numeric', StandardScaler(), dnn_numeric_cols),
    ]
)

X_train_dnn = dnn_preprocessor.fit_transform(train_df[dnn_feature_cols])
X_val_dnn = dnn_preprocessor.transform(val_df[dnn_feature_cols])
X_test_dnn = dnn_preprocessor.transform(test_df[dnn_feature_cols])

y_train = train_df[target_col].to_numpy(dtype=np.float32)
y_val = val_df[target_col].to_numpy(dtype=np.float32)
y_test = test_df[target_col].to_numpy(dtype=np.float32)

X_train_dnn = X_train_dnn.astype(np.float32)
X_val_dnn = X_val_dnn.astype(np.float32)
X_test_dnn = X_test_dnn.astype(np.float32)

print('DNN feature dimension:', X_train_dnn.shape[1])

In [ ]:
BATCH_SIZE = 256

train_dnn_ds = (
    tf.data.Dataset.from_tensor_slices((X_train_dnn, y_train))
    .shuffle(buffer_size=len(X_train_dnn), seed=RANDOM_SEED, reshuffle_each_iteration=True)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

val_dnn_ds = tf.data.Dataset.from_tensor_slices((X_val_dnn, y_val)).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_dnn_ds = tf.data.Dataset.from_tensor_slices((X_test_dnn, y_test)).batch(BATCH_SIZE)

In [ ]:
def build_dnn_model(input_dim: int) -> keras.Model:
    inputs = keras.Input(shape=(input_dim,), name='tabular_features')
    x = keras.layers.Dense(256, activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=RANDOM_SEED))(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.Dense(128, activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=RANDOM_SEED))(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.Dense(64, activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=RANDOM_SEED))(x)
    outputs = keras.layers.Dense(1, kernel_initializer=keras.initializers.HeNormal(seed=RANDOM_SEED), name='turnover')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name='DeepDNN')
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss='mse',
        metrics=[
            keras.metrics.MeanSquaredError(name='mse'),
            keras.metrics.RootMeanSquaredError(name='rmse'),
            keras.metrics.MeanAbsoluteError(name='mae'),
            keras.metrics.MeanAbsolutePercentageError(name='mape'),
        ],
    )
    return model

dnn_model = build_dnn_model(X_train_dnn.shape[1])

dnn_callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-5),
]

dnn_history = dnn_model.fit(
    train_dnn_ds,
    validation_data=val_dnn_ds,
    epochs=60,
    verbose=2,
    callbacks=dnn_callbacks,
)

In [ ]:
dnn_eval = dnn_model.evaluate(test_dnn_ds, return_dict=True, verbose=0)
dnn_preds = dnn_model.predict(X_test_dnn, batch_size=BATCH_SIZE, verbose=0).squeeze()

print('DNN test metrics:', dnn_eval)


def compute_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
    mse = mean_squared_error(y_true, y_pred)
    rmse = float(np.sqrt(mse))
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'R2': r2}

dnn_metrics = compute_metrics(y_test, dnn_preds)
dnn_metrics

In [ ]:
pd.DataFrame(dnn_history.history)[['loss', 'val_loss']].plot(figsize=(6, 4), title='DNN loss')
plt.ylabel('MSE')
plt.show()

## Model 2: GRU-based sequence-to-sequence network

In [ ]:
WINDOW_SIZE = 12
HORIZON = 1
seq_feature_cols = ['TurnoverScaled', 'MonthSin', 'MonthCos', 'YearScaled', 'SeriesIndexNorm']
seq_target_col = 'TurnoverScaled'


def build_sequence_arrays(df: pd.DataFrame, feature_cols: list[str], target_col: str, window: int, horizon: int):
    sequences = []
    targets = []
    target_actuals = []

    for _, group in df.groupby('Series ID'):
        group = group.sort_values('Month')
        feature_data = group[feature_cols].to_numpy(dtype=np.float32)
        target_data = group[target_col].to_numpy(dtype=np.float32)
        actual_data = group['Turnover'].to_numpy(dtype=np.float32)
        n_samples = len(group) - window - horizon + 1
        if n_samples <= 0:
            continue
        for idx in range(n_samples):
            sequences.append(feature_data[idx: idx + window])
            targets.append(target_data[idx + window: idx + window + horizon])
            target_actuals.append(actual_data[idx + window: idx + window + horizon])

    if not sequences:
        return (
            np.empty((0, window, len(feature_cols)), dtype=np.float32),
            np.empty((0, horizon), dtype=np.float32),
            np.empty((0, horizon), dtype=np.float32),
        )

    return (np.stack(sequences), np.stack(targets), np.stack(target_actuals))


X_seq_train, y_seq_train, _ = build_sequence_arrays(train_df, seq_feature_cols, seq_target_col, WINDOW_SIZE, HORIZON)
X_seq_val, y_seq_val, _ = build_sequence_arrays(val_df, seq_feature_cols, seq_target_col, WINDOW_SIZE, HORIZON)
X_seq_test, y_seq_test_scaled, y_seq_test_actual = build_sequence_arrays(test_df, seq_feature_cols, seq_target_col, WINDOW_SIZE, HORIZON)

print('Sequence shapes -> train:', X_seq_train.shape, 'val:', X_seq_val.shape, 'test:', X_seq_test.shape)

In [ ]:
SEQ_BATCH_SIZE = 128

train_seq_ds = (
    tf.data.Dataset.from_tensor_slices((X_seq_train, y_seq_train))
    .shuffle(buffer_size=len(X_seq_train), seed=RANDOM_SEED, reshuffle_each_iteration=True)
    .batch(SEQ_BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

val_seq_ds = tf.data.Dataset.from_tensor_slices((X_seq_val, y_seq_val)).batch(SEQ_BATCH_SIZE).prefetch(AUTOTUNE)
test_seq_ds = tf.data.Dataset.from_tensor_slices((X_seq_test, y_seq_test_scaled)).batch(SEQ_BATCH_SIZE)

In [ ]:
def build_gru_model(window: int, feature_dim: int, horizon: int) -> keras.Model:
    inputs = keras.Input(shape=(window, feature_dim), name='sequence_features')
    x = keras.layers.GRU(128, return_sequences=True, kernel_initializer='glorot_uniform')(inputs)
    x = keras.layers.LayerNormalization()(x)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.GRU(64, return_sequences=False, kernel_initializer='glorot_uniform')(x)
    x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(horizon, kernel_initializer='glorot_uniform', name='scaled_turnover')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name='GRUSeq2Seq')
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss='mse',
        metrics=[
            keras.metrics.MeanSquaredError(name='mse'),
            keras.metrics.RootMeanSquaredError(name='rmse'),
            keras.metrics.MeanAbsoluteError(name='mae'),
            keras.metrics.MeanAbsolutePercentageError(name='mape'),
        ],
    )
    return model

gru_model = build_gru_model(WINDOW_SIZE, X_seq_train.shape[2], HORIZON)

gru_callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-5),
]

gru_history = gru_model.fit(
    train_seq_ds,
    validation_data=val_seq_ds,
    epochs=80,
    verbose=2,
    callbacks=gru_callbacks,
)

In [ ]:
gru_eval = gru_model.evaluate(test_seq_ds, return_dict=True, verbose=0)
gru_preds_scaled = gru_model.predict(X_seq_test, batch_size=SEQ_BATCH_SIZE, verbose=0)
gru_preds = turnover_scaler.inverse_transform(gru_preds_scaled)

y_seq_test_actual = y_seq_test_actual.squeeze(axis=-1)
gru_preds = gru_preds.squeeze(axis=-1)

print('GRU test metrics:', gru_eval)

gru_metrics = compute_metrics(y_seq_test_actual, gru_preds)
gru_metrics

In [ ]:
pd.DataFrame(gru_history.history)[['loss', 'val_loss']].plot(figsize=(6, 4), title='GRU loss')
plt.ylabel('MSE')
plt.show()

## Model comparison

In [ ]:
comparison_df = pd.DataFrame(
    [
        {'Model': 'Deep Neural Network', 'Samples': len(y_test), **dnn_metrics},
        {'Model': 'GRU Seq2Seq', 'Samples': len(y_seq_test_actual), **gru_metrics},
    ]
).set_index('Model')
comparison_df.sort_values('RMSE')

Both trained models meet the baseline requirement for Assignment 2 Task 1. Subsequent tasks will fine-tune hyperparameters and expand the evaluation and discussion sections.